In [1]:
import os
import h5py
import numpy as np
import matplotlib.pyplot as plt
import sys
import importlib
from astropy.cosmology import FlatLambdaCDM

from utils_orbs.orb_paths import SetupPaths
import utils_orbs.readsubfHDF5Py3 as readSub
from utils_orbs.read_group_cats import ReadCats
from utils_orbs.merger_trees import TraceMergerTree
from utils_orbs.readMergerHistoryFiles import FindHistory
from utils_orbs.vectorCorrection import vectorCorrection as vector

paths = SetupPaths()

imported h5py...
modules seem to have loaded?


In [2]:
plt.show();
plt.rcParams.update({'font.size':16,"xtick.direction":"in","ytick.direction":"in", 
                     "xtick.top":True, "ytick.right":True,"text.usetex":False,
                     "xtick.labelsize":18,"ytick.labelsize":18})

# Definitions for this notebook:

In [3]:
z_directory = {"33":"z=2","40":"z=1.5","50":"z=1"}

## Load in data from hydro catalog

In [5]:
# load in z=1.5 cat
catpath = paths.path_tnghydro
keysel_default =  [
            'GroupPos','Group_M_TopHat200', 
            'Group_R_TopHat200','GroupNsubs',
            'GroupFirstSub',
            'SubhaloGrNr','SubhaloMass',
            'SubhaloPos','SubhaloVel' ]
    
catalog = readSub.subfind_catalog(
            basedir=catpath,
            snapnum=40,
            keysel=keysel_default
            )

In [14]:
catalog.SubhaloPos[222536]

array([32281.42578125, 37403.015625  , 42837.59765625])

## Load in data from orbit data

In [6]:
orbit_dict = {}
for snap in [33,40,50]:
    f = h5py.File(f"{paths.path_data}orbitdata_{snap}.hdf5",'r')
    orbit_dict[str(snap)] = {}
    for key, val in f.items():
        orbit_dict[str(snap)][key] = np.array(val)
    f.close()

f = h5py.File(f"{paths.path_data}snapshot_data.hdf5",'r')
snap_dict = {}
for key, val in f.items():
    snap_dict[key] = np.array(val)
f.close()

In [7]:
print(f"Subfind ID 1: {orbit_dict['40']['SubfindID1'][0]}")
print(f"Subfind ID 2: {orbit_dict['40']['SubfindID2'][0]}")

Subfind ID 1: 222536.0
Subfind ID 2: 222537.0


In [8]:
loc = np.where(snap_dict['Snapshot'][::-1][:-1]==40)[0]
print(f"Separation: {orbit_dict['40']['Separations'][0][loc[0]]} kpc") 
print(f"Velocity: {orbit_dict['40']['Velocities'][0][loc[0]]} km/s")

Separation: 185.53270051150062 kpc
Velocity: 145.11892700195312 km/s


## Load in data from pair catalog

In [15]:
f = h5py.File(f"../data/lowmass_40.hdf5", 'r')
pairs = {}
for key, val in f.items():
    if key != "Header":
        pairs[key] = np.array(val) 
f.close()

In [29]:
little_h  = 0.6774
scale = 1/(1+catalog.redshift)

pairs['Sub1 Pos'][0]

array([19096.23824094, 22125.94022781, 25340.79430247])

In [41]:
# check that masses are equal:
catmass = catalog.SubhaloMass[222536]
samplemass = pairs['Sub1 Mass'][0]
if catmass/little_h == samplemass:
    print("Masses are equal (in physical units)")
else:
    print("Masses are unequal")


Masses are equal (in physical units)


In [43]:
# check that positions are equal:
catpos = catalog.SubhaloPos[222536]
samplepos = pairs['Sub1 Pos'][0]
if (scale*catpos/little_h == samplepos).all:
    print("Physical positions are equal (in physical units)")
else:
    print("Positions are unequal")

Physical positions are equal (in physical units)


In [34]:
comov1 = catalog.SubhaloPos[222536]
comov2 = catalog.SubhaloPos[222537]
comovdist = np.linalg.norm(vector(comov1,comov2,75000))

phys1 = pairs['Sub1 Pos'][0]
phys2 = pairs['Sub2 Pos'][0]
physdist = np.linalg.norm(vector(phys1,phys2,75000))

In [45]:
if round(scale*comovdist/little_h,4) == round(physdist,4):
    print("The physical separations are equal")
else:
    print("Physical separations differ")


The physical separations are equal


Units look correct, so I will be able to just divide the physical distance that I have by the scale and multiply by little h to get the units in ckpc/h! 